In [1]:
import psycopg2
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import sys
sys.path.append('../')

from src.pySQL_funcs import *

In [2]:
#FOR SECURITY I EXCLUDE THE AWS ADDRESS AND PASSWORD HERE
#SYNTAX IS "dbname= user= host= port=5432 password="
with open('../src/pw') as pw_file:
    pw = pw_file.readline()
    
conn = psycopg2.connect(pw)
cur = conn.cursor()

In [3]:
# Example Query to test connection
query = """
SELECT * FROM pokemon LIMIT 10;
"""
#cur.execute(query)
#cur.fetchall()
pretty_query(cur, query, conn)

,id,name
0,0,Gliscor
1,1,Fletchinder
2,2,Poliwrath
3,3,Cresselia
4,4,Frogadier
5,5,Sceptile
6,6,Lapras
7,7,Sharpedo
8,8,Slowbro
9,9,Omastar


In [4]:
## Find the 10 most popular abilities for pokemon movesets
query = """
SELECT n.name, SUM(count) AS total_number_pokemon_used
  FROM battle_abilities b 
  JOIN abilities n ON b.ability_id=n.id 
  GROUP BY n.name ORDER BY COUNT(*) DESC LIMIT 10;
"""
pretty_query(cur, query, conn)

,name,total_number_pokemon_used
0,levitate,75154300.0
1,innerfocus,8890850.0
2,intimidate,64265100.0
3,waterabsorb,15057900.0
4,regenerator,44654700.0
5,pressure,28741300.0
6,technician,27212000.0
7,sturdy,20654900.0
8,synchronize,16686100.0
9,flashfire,32312100.0


In [5]:
## Find the 10 most popular pokemon used as a percent of month 
query = """
SELECT p.name, usage, month 
  FROM battles b 
  JOIN pokemon p ON b.id=p.id
 ORDER BY usage DESC LIMIT 10;
"""
pretty_query(cur, query, conn)

,name,usage,month
0,Dragapult,0.375045,01-2020
1,Dragapult,0.365004,12-2019
2,Dragapult,0.345182,02-2020
3,Cinderace,0.315563,06-2020
4,Landorus-Therian,0.311849,06-2019
5,Darmanitan-Galar,0.310821,12-2019
6,Landorus-Therian,0.303670,11-2017
7,Landorus-Therian,0.301281,12-2017
8,Dragapult,0.298842,03-2020
9,Landorus-Therian,0.297543,02-2019


In [6]:
#Find Pokemon with the most months on as number 1 in usage
query = """
SELECT name, COUNT(*) AS months_on_top
  FROM (
        SELECT p.name, b.month, 
               RANK() OVER(PARTITION BY month ORDER BY usage DESC) r
          FROM battles b 
          JOIN pokemon p ON b.id=p.id 
       ) monthly_rank
  WHERE r=1
  GROUP BY name
 ORDER BY months_on_top DESC LIMIT 10;
"""
pretty_query(cur, query, conn)

,name,months_on_top
0,Landorus-Therian,46
1,Dragapult,4
2,Garchomp,3
3,Gengar,3
4,Clefable,2
5,Cinderace,1


In [7]:
cur.close()  # Close the cursor
conn.close()